# SI: On the Molecular Anion-Cation Contrast of Caffeine Solvation in Salt Solutions.

Stefan Hervø-Hansen<sup>a</sup>, Jan Heyda<sup>b</sup> Mikael Lund<sup>a</sup>, and Nobuyuki Matubayasi<sup>c</sup><br><br>
<sup>a</sup> Division of Theoretical Chemistry, Department of Chemistry, Lund University, Lund SE 221 00, Sweden.
<br> <sup>b</sup> Department of Physical Chemistry, University of Chemistry and Technology, Prague CZ-16628, Czech Republic.
<br> <sup>c</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonak
a, Osaka 560-8531, Japan.<br>
To whom correspondence may be addressed: stefan.hervo_hansen@teokem.lu.se, mikael.lund@teokem.lu.se, and nobuyuki@cheng.es.osaka-u.ac.jp.

# Part 1: Simulations

## Introduction

We present a study of the solvation free energy of caffeine in electrolyte solutions using the theory of energy-representation in combination with all-atom simulations. In specific we desire to calculate Setschenow (Sechenov) coefficients, $k_s$,[<sup>1</sup>](#fn1) for various salts. The Setschenow equation[<sup>1</sup>](#fn1) is on the $\ln$-scale given as
$$ \ln \left( \frac{S}{S_0}\right) =  -k_sc_s = \ln\gamma = \frac{\Delta\mu^{ex}}{kT} \tag 1$$
where $S$ and $S_0$ are solubilities in pure water and an electrolyte solution of concentration $c_s$. In literature eq. 1 is most commonly presented on $\log_{10}$ scale, however on the presented form, the Setschenow equation shows its clear relationship to the activity coefficient and chemical potential. In this notebook we will conduct all atomic molecular dynamics simulations.

### Methods & Materials
Molecular dynamics simulations are conducted using the openMM (7.4.0)[<sup>2</sup>](#fn2) software package modded with the openmmtools[<sup>3</sup>](#fn3) and parmed[<sup>4</sup>](#fn4) packages. For the simulation of caffeine a GROMOS (ffGF53a6) derived Kirkwood-Buff force field with adjustments to the partial charges and geometrical parameters, which has previously been able able to reproduce experimental solubilities of caffeine in water and urea solutions[<sup>5</sup>](#fn5) was employed with the SPC/E[<sup>6</sup>](#fn6) force field employed for water and optimized ion parameters for alkali cations and halide anions[<sup>7</sup>](#fn7). The isothermal-isobaric ensemble will be sampled using a combination of a geodesic Langevin integrator[<sup>8</sup>](#fn8) and a Monte Carlo barostat[<sup>9,</sup>](#fn9)[<sup>10</sup>](#fn10). The trajectories will be analyzed using MDtraj[<sup>11</sup>](#fn11) for structural properties, while ERmod[<sup>12</sup>](#fn12) be utilized for the calculation of solvation free energies and can be found in the [Part 2 Jupyter notebook](Analysis.ipynb).


### References
1. <span id="fn1"> Setschenow J (1889) Über die Konstitution der Salzlösungen auf Grund ihres Verhaltens zu Kohlensäure. Zeitschrift für Physikalische Chemie 4U(1):117–125.
2. <span id="fn1"> Eastman P, et al. (2017) OpenMM 7: Rapid development of high performance algorithms for molecular dynamics. PLOS Computational Biology 13(7):e1005659.</span><br>
3. <span id="fn2"> https://github.com/choderalab/openmmtools</span><br>
4. <span id="fn3"> https://github.com/ParmEd/ParmEd </span><br>
5. <span id="fn4"> Sanjeewa R, Weerasinghe S (2010) Development of a molecular mechanics force field for caffeine to investigate the interactions of caffeine in different solvent media. Journal of Molecular Structure: THEOCHEM 944(1–3):116–123. </span><br>
6. <span id="fn5"> Berendsen HJC, Grigera JR, Straatsma TP (1987) The missing term in effective pair potentials. The Journal of Physical Chemistry 91(24):6269–6271. </span><br>
7. <span id="fn6"> Heyda J, Vincent JC, Tobias DJ, Dzubiella J, Jungwirth P (2010) Ion Specificity at the Peptide Bond: Molecular Dynamics Simulations of N-Methylacetamide in Aqueous Salt Solutions. The Journal of Physical Chemistry B 114(2):1213–1220. </span><br>
8. <span id="fn8"> Leimkuhler B, Matthews C (2016) Efficient molecular dynamics using geodesic integration and solvent–solute splitting. Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences 472(2189):20160138. </span><br>
9. <span id="fn9"> Chow K-H, Ferguson DM (1995) Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91(1–3):283–289. </span><br>
10. <span id="fn10"> Åqvist J, Wennerström P, Nervall M, Bjelic S, Brandsdal BO (2004) Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384(4–6):288–294. </span><br>
11. <span id="fn11"> McGibbon RT, et al. (2015) MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109(8):1528–1532. </span><br>
12. <span id="fn12"> Sakuraba S, Matubayasi N (2014) Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. Journal of Computational Chemistry 35(21):1592–1608. </span><br>

### Import of Python Modules & Definition of Auxiliary Functions

In [ ]:
# Notebook dependent libs
import parmed as pmd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import mdtraj as md
from uncertainties import unumpy
import os, time, re, yaml, string
from scipy.stats import linregress

# Simulation specific libs
import sys
from simtk.openmm import app
import simtk.openmm as mm
import openmmtools as mmtools
import parmed as pmd
from mdtraj.reporters import XTCReporter

homedir='/home/stefan/Caffeine_solubility'
#homedir = !pwd

## Molecular Dynamics Simulations

### Simulation settings
For the calculation of solvation free energies, we need to simulate the solvated state ($\lambda=1$) and the reference state ($\lambda=0$), with the duration of the simulation determined by together with the output frequency for the saving configurations and other thermodynamic properties for statistical evaluation, all of which is determined by the `states` variable. The molecular dynamics simulations have been set up such that one can run `Nruns` independent molecular dynamics simulations. The system composition of salts and their concentrations are controlled by the variables `salts` and `salt_concentrations` respectively.

In [ ]:
###############################################################
#                 CHOOSE YOUR FORCE FIELD                     #
# False: KB-force field of caffeine (see Methods & Materials) #
# True:  Uses OPLS force field                                #
WANT_OPLS = False                                             #
###############################################################

# State of simulations, (outFreq is steps per frame)
Nruns = 25
states = {                                                               
          'sol': {'Nsteps': 1000000,  'OutFreq': 50}, # 2 nanoseconds, 20000 frames      (x25 50 ns)
          'ref': {'Nsteps': 200000,  'OutFreq': 500} # 0.4 nanoseconds, 400 frames       (x25 10 ns)
         }

salts = {'NaCl' : {'Cation': 'Na', 'Anion': 'Cl' },
#         'NaI'  : {'Cation': 'Na', 'Anion': 'I'  },
#         'NaF'  : {'Cation': 'Na', 'Anion': 'F'  },
#         'KCl'  : {'Cation': 'K' , 'Anion': 'Cl' },
#         'KI'   : {'Cation': 'K' , 'Anion': 'I'  },
#         'KF'   : {'Cation': 'K' , 'Anion': 'F'  },
#         'CsCl' : {'Cation': 'Cs', 'Anion': 'Cl' },
#         'CsI'  : {'Cation': 'Cs', 'Anion': 'I'  },
#         'CsF'  : {'Cation': 'Cs', 'Anion': 'F'  } 
        }

# Approximate concentrations of salts with
salt_concentrations = {
#                       0.00: {'Caffeine': 1, 'Water': 1000, 'Cation':0,  'Anion':0},
#                       0.25: {'Caffeine': 1, 'Water': 1000, 'Cation':5,  'Anion':5},
                       0.50: {'Caffeine': 1, 'Water': 1000, 'Cation':9,  'Anion':9},
#                       0.75: {'Caffeine': 1, 'Water': 1000, 'Cation':14, 'Anion':14},
                       1.00: {'Caffeine': 1, 'Water': 1000, 'Cation':18, 'Anion':18},
#                       1.25: {'Caffeine': 1, 'Water': 1000, 'Cation':23, 'Anion':23},   
                       1.50: {'Caffeine': 1, 'Water': 1000, 'Cation':27, 'Anion':27},   
#                       1.75: {'Caffeine': 1, 'Water': 1000, 'Cation':32, 'Anion':32},
                       2.00: {'Caffeine': 1, 'Water': 1000, 'Cation':36, 'Anion':36}   
                      }

### Construction of topology (.top) and structure (.pdb) files
Fully automated construction of topologies files in gromacs format and initial configurations using packmol. No major important parameters to edit in the following code.

In [ ]:
packmol_script="""
tolerance 2.0
filetype pdb
output Caffeine_{cation}{anion}_sol_{run}.pdb
add_box_sides 0.5

structure ../../../../PDB_files/single-caffeine-molecule.pdb
        number {N_caffeine}
        fixed 16. 16. 16. 0. 0. 0.
        centerofmass
end structure

{salt}structure ../../../../PDB_files/{anion}.pdb
{salt}        number {N_anion}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

{salt}structure ../../../../PDB_files/{cation}.pdb
{salt}        number {N_cation}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

structure ../../../../PDB_files/water.pdb
        number {N_wat}
        inside cube 0. 0. 0. 32.
end structure
"""

topology="""
[ system ]
; Name
Caffeine in {cation}{anion} {conc} M aqueous solution.

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               yes             0.5     0.8333

; Include all atomtypes
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_spc.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_ions.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_caffeine.itp"

; Include all topologies
#include "/home/stefan/Caffeine_solubility/force_fields/ions.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/spce.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/caffeine-KBFF.itp"

[ molecules ]
; Compound         #mols
2S09               {N_caffeine}
{salt}{anion}            {N_anion}
{salt}{cation}           {N_cation}
SOL                {N_wat}
"""

if WANT_OPLS:
    packmol_script="""
tolerance 2.0
filetype pdb
output Caffeine_{cation}{anion}_sol_{run}.pdb
add_box_sides 0.5

structure ../../../../../force_fields/opls_ff/SAMPL5_080A_caffeine.pdb
        number {N_caffeine}
        fixed 16. 16. 16. 0. 0. 0.
        centerofmass
end structure

{salt}structure ../../../../../force_fields/opls_ff/{anion}.pdb
{salt}        number {N_anion}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

{salt}structure ../../../../../force_fields/opls_ff/{cation}.pdb
{salt}        number {N_cation}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

structure ../../../../../force_fields/opls_ff/water.pdb
        number {N_wat}
        inside cube 0. 0. 0. 32.
end structure
"""

    topology="""
[ system ]
; Name
Caffeine in {cation}{anion} {conc} M aqueous solution (OPLS FF).

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               yes             0.5     0.5

; Includes
#include "/home/stefan/Caffeine_solubility/force_fields/opls_ff/ffnonbonded.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/opls_ff/ffbonded.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/opls_ff/SAMPL5_080A_caffeine.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/opls_ff/spce.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/opls_ff/ions.itp"

[ molecules ]
; Compound         #mols
UNK               {N_caffeine}
{salt}{anion}            {N_anion}
{salt}{cation}           {N_cation}
SOL                {N_wat}
"""

In [ ]:
%cd -q $homedir
for saltdir, salt in salts.items():
    for concentration, Nparticles in salt_concentrations.items():
        conc = '{0:.2f}'.format(concentration)
        for run in range(Nruns):
            if WANT_OPLS:
                %cd -q $homedir/Simulations/opls_simulations/$saltdir/$conc/$run
            else:
                %cd -q $homedir/Simulations/$saltdir/$conc/$run
            # Packmol Input
            with open('packmol.in', 'w') as text_file:
                # fix for no salt:
                if concentration:
                    saltFix=''
                else:
                    saltFix='#'
                text_file.write(packmol_script.format(N_caffeine=Nparticles['Caffeine'], N_wat=Nparticles['Water'],
                                                      N_cation=Nparticles['Cation'], N_anion=Nparticles['Anion'],
                                                      salt=saltFix, run=run, cation=salt['Cation'],
                                                      anion=salt['Anion']))
            !packmol < packmol.in
    
            # Topology input
            if run == 0:
                with open('../Caffeine_{salt}_sol.top'.format(salt=saltdir), 'w') as text_file:
                    # fix for no salt:
                    if concentration:
                        saltFix=''
                    else:
                        saltFix=';'
                    if WANT_OPLS:
                        text_file.write(topology.format(conc=concentration, salt=saltFix,
                                                        N_caffeine=Nparticles['Caffeine'], N_wat=Nparticles['Water'],
                                                        N_cation=Nparticles['Cation'], N_anion=Nparticles['Anion'],
                                                        cation=salt['Cation'].upper(), anion=salt['Anion'].upper()))
                    else:
                        text_file.write(topology.format(conc=concentration, salt=saltFix,
                                                        N_caffeine=Nparticles['Caffeine'], N_wat=Nparticles['Water'],
                                                        N_cation=Nparticles['Cation'], N_anion=Nparticles['Anion'],
                                                        cation=salt['Cation'], anion=salt['Anion']))
        
            # Solvated state
            mol = pmd.load_file('../Caffeine_{salt}_sol.top'.format(salt=saltdir),
                                xyz='Caffeine_{salt}_sol_{run}.pdb'.format(salt=saltdir, run=run))
            if run == 0:
                mol.save('../Caffeine_{salt}_sol.top'.format(salt=saltdir), overwrite=True)
        
            # Reference state
            if WANT_OPLS:
                mol.strip(':UNK')
            else:
                mol.strip(':2S09')
            if run == 0:
                mol.save('../Caffeine_{salt}_ref.top'.format(salt=saltdir, overwrite=True))
            mol.save('Caffeine_{salt}_ref_{run}.pdb'.format(salt=saltdir, run=run)) 
        
        print('Wrote initial configurations and topology files to'+os.getcwd())

### Simulation setup using OpenMM
Fully automated construction of simulation input files using python API for OpenMM. In the following one can edit the integration scheme and its parameters set in the variable `integrator` as well as editing the barostat as currently determined from `mm.MonteCarloBarostat`. Additionally one may change the non-bonded methods and their cutoffs in the `system` variable with the option of adding Lennard-Jones switching functions via the `forces` variable. Finally one may edit the number of minimization (`sim.minimizeEnergy`) and equilibration steps conducted as well as choosing whether the simulation should be conducted on GPUs or CPUs via the `platform` and `properties` variables.

In [ ]:
%cd -q $homedir
N_simulations = 0
for saltdir, salt in salts.items():
    for concentration in salt_concentrations:
        conc = '{0:.2f}'.format(concentration)
        for run in range(Nruns):
            if WANT_OPLS:
                %cd -q $homedir/Simulations/opls_simulations/$saltdir/$conc/$run
            else:
                %cd -q $homedir/Simulations/$saltdir/$conc/$run
            for state, settings in states.items():
                openmm_script="""

# Imports
import sys
import os
from simtk.openmm import app
import simtk.openmm as mm
import openmmtools as mmtools
from parmed import load_file, unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
init_conf = load_file('../Caffeine_{salt}_{state}.top', xyz='Caffeine_{salt}_{state}_{run}.pdb')

# Creating system
print('Creating OpenMM System')
system = init_conf.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.00001,
                                nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds)

# Calculating total mass of system
total_mass = 0
for i in range(system.getNumParticles()):
    total_mass += system.getParticleMass(i).value_in_unit(u.dalton)
total_mass *= u.dalton

# Temperature-coupling by geodesic Langevin integrator (NVT)
integrator = mmtools.integrators.GeodesicBAOABIntegrator(K_r = 3,
                                                         temperature = 298.15*u.kelvin,
                                                         collision_rate = 1.0/u.picoseconds,
                                                         timestep = 2.0*u.femtoseconds
                                                        )

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

# Add LJ switching functions
forces = {{system.getForce(index).__class__.__name__: 
          system.getForce(index) for index in range(system.getNumForces())}}
forces['CustomNonbondedForce'].setUseSwitchingFunction(True)
forces['CustomNonbondedForce'].setSwitchingDistance(1*u.nanometer)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(init_conf.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(init_conf.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=500000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(50000)  # 50000*2 fs = 0.1 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output_{state}.dat', {outFreq}, totalSteps={Nsteps}+50000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory_{state}.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS (Production run)
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""
                with open('openMM_{state}.py'.format(state=state), 'w') as text_file:
                    text_file.write(openmm_script.format(state=state, Nsteps=settings['Nsteps'], outFreq=settings['OutFreq'],
                                                         run=run, salt=saltdir))
                N_simulations+=1
                print('Wrote run_openMM.py files to '+os.getcwd())

    
print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script
Submit script for servers employing job scheduling. The below example is utilizing PBS (for a quick guide see [here](https://latisresearch.umn.edu/creating-a-PBS-script)). However the code below may be edited to utilize Slurm instead (documentation [here](https://slurm.schedmd.com)) by changing the variable `submit_script` and by executing the commands `!sbatch submit_sol.pbs` and `!sbatch submit_ref.pbs` instead of `qsub`.

In [ ]:
for saltdir, salt in salts.items():
    for concentration in salt_concentrations:
        conc = '{0:.2f}'.format(concentration)
        for run in range(Nruns):
            if WANT_OPLS:
                %cd -q $homedir/Simulations/opls_simulations/$saltdir/$conc/$run
            else:
                %cd -q $homedir/Simulations/$saltdir/$conc/$run
            for state in states:
                submit_script="""#!/bin/bash
#PBS -l nodes=1:ppn=18:gpu:gpus=1     # 1 node, 18 cores, GPU node, 1 gpu.
#PBS -N {conc}_M_{salt}_{state}_{run}    # Name of job
#PBS -e run_{state}.err               # error output
#PBS -o run_{state}.out               # output file name

source ~/.bashrc
source ~/.bash_profile
cd {path}

python openMM_{state}.py"""

                with open('submit_{state}.pbs'.format(state=state), 'w') as text_file:
                    text_file.write(submit_script.format(conc=conc, state=state, run=run,
                                                     path=os.getcwd(), salt=saltdir))
            !qsub submit_sol.pbs
            time.sleep(1) # Safety in submission of jobs: can cause problems if too fast
            !qsub submit_ref.pbs
            time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

### Trajectory processing
Fully automated code to construct a single large trajectory file from the `Nruns` independent simulations.<br>
<em><strong>Warning</strong>: This cell is time, memory, and storage consuming.</em>

In [ ]:
for saltdir, salt in salts.items():
    for concentration in salt_concentrations:
        for state in states:
            conc = '{0:.2f}'.format(concentration)
            if WANT_OPLS:
                %cd -q $homedir/Simulations/opls_simulations/$saltdir/$conc
            else:
                %cd -q $homedir/Simulations/$saltdir/$conc
        
            for run in range(Nruns):
                if run == 0:
                    traj = md.load_xtc('0/trajectory_{state}.xtc'.format(state=state),
                                       top='0/Caffeine_{salt}_{state}_0.pdb'.format(salt=saltdir, state=state))
                    continue
                traj_app = md.load_xtc('{run}/trajectory_{state}.xtc'.format(run=run, state=state),
                                       top='{run}/Caffeine_{salt}_{state}_{run}.pdb'.format(state=state, run=run, salt=saltdir))
                traj = md.join([traj, traj_app], check_topology=True, discard_overlapping_frames=False)
            
            traj.save_xtc('trajectory_{state}_full.xtc'.format(state=state), force_overwrite=False)